In [2]:
import pandas as pd 
import geopandas as gpd
import os

In [3]:
nome_comune="reggiocalabria"
folder_head="C:\\Users\\gabri\\database_vari_miei\\replication_politics_results\\"

In [4]:
folder_dati="C:\\Users\\gabri\\database_vari_miei\\replication_politics_results\\reggiocalabria\\" 
url_geojson = folder_head+"geoframes\\"+nome_comune+"\\precincts_"+nome_comune+"_census"
url_geojson_buildings = folder_head+"geoframes\\"+ f'{nome_comune}\\precincts_{nome_comune}_bulding.geojson'

In [5]:
geo = gpd.read_file(url_geojson_buildings)

In [6]:
df = pd.read_csv(folder_dati+os.listdir(folder_dati)[1])

In [7]:
dplot = pd.merge(geo,df)

In [8]:
dplot.drop(['COD_REG', 'COD_ISTAT', 'PRO_COM', 'SEZ2011',
       'SEZ', 'COD_STAGNO', 'COD_FIUME', 'COD_LAGO', 'COD_LAGUNA', 'COD_VAL_P',
       'COD_ZONA_C', 'COD_IS_AMM', 'COD_IS_LAC', 'COD_IS_MAR', 'COD_AREA_S',
       'COD_MONT_D', 'LOC2011', 'COD_LOC', 'TIPO_LOC', 'COM_ASC', 'COD_ASC',
       'ACE', 'Shape_Leng'],axis=1,inplace=True,errors="ignore")

In [ ]:
import geopandas as gpd

# gdf = ...  # your GeoDataFrame

# --- 1) Compute vote shares from absolute counts ---
# Avoid division by zero
gdf = dplot.copy()
den = gdf["TOTALE"].replace({0: None})
gdf["AFFLUENZA"] = gdf["AFFLUENZA"]*100

gdf["TRIDICO_share"]  = (gdf["TRIDICO"]  / den) * 100
gdf["OCCHIUTO_share"] = (gdf["OCCHIUTO"] / den) * 100
gdf["TOSCANO_share"]  = (gdf["TOSCANO"]  / den) * 100

# Optional: if AFFLUENZA is in 0–1 rather than 0–100, uncomment:
# if gdf["AFFLUENZA"].max() <= 1.0:
#     gdf["AFFLUENZA"] = gdf["AFFLUENZA"] * 100

# Nice formatted fields for hover (tooltips)
gdf["TRIDICO_%"]  = gdf["TRIDICO_share"].map(lambda x: f"{x:.2f}%" if x is not None else "NA")
gdf["OCCHIUTO_%"] = gdf["OCCHIUTO_share"].map(lambda x: f"{x:.2f}%" if x is not None else "NA")
gdf["TOSCANO_%"]  = gdf["TOSCANO_share"].map(lambda x: f"{x:.2f}%" if x is not None else "NA")

# If you also want affluenza formatted:
gdf["AFFLUENZA_%"] = gdf["AFFLUENZA"].map(lambda x: f"{x:.2f}%" if x is not None else "NA")


# --- 2) Map A: AFFLUENZA with viridis + hover showing vote shares ---
m_affluenza = gdf.explore(
    column="AFFLUENZA",
    cmap="viridis",
    legend=True,
    tooltip=["SEZIONE", "AFFLUENZA_%", "TRIDICO_%", "OCCHIUTO_%", "TOSCANO_%"],
    tooltip_kwds={"sticky": True},
    # optional styling
    style_kwds={"weight": 0.6, "fillOpacity": 0.75},
        tiles="CartoDB positron"
)

# --- 3) Map B: TRIDICO vote share with Reds + hover showing vote shares ---
m_tridico = gdf.explore(
    column="TRIDICO_share",
    cmap="Reds",
    legend=True,
    tooltip=["SEZIONE", "TRIDICO_%", "OCCHIUTO_%", "TOSCANO_%","AFFLUENZA_%"],
    tooltip_kwds={"sticky": True},
    style_kwds={"weight": 0.8, "fillOpacity": 0.5},
    tiles="CartoDB positron"
)

# In notebooks, just display `m_affluenza` or `m_tridico`.
# To save as HTML:
m_affluenza.save("mappe\\affluenza_map.html")
m_tridico.save("mappe\\tridico_share_map.html")


# --- 4) Compute margin (Tridico vs Occhiuto) ---
gdf["TRI_vs_OCC_margin"] = gdf["TRIDICO_share"] - gdf["OCCHIUTO_share"]

# formatted margin for tooltip
gdf["TRI_vs_OCC_margin_%"] = gdf["TRI_vs_OCC_margin"].map(
    lambda x: f"{x:+.2f}%" if x is not None else "NA"
)

# important: symmetric color scale around 0
max_abs = abs(gdf["TRI_vs_OCC_margin"]).max()

# --- 5) Map C: Tridico vs Occhiuto (Red ↔ Blue) ---
m_margin = gdf.explore(
    column="TRI_vs_OCC_margin",
    cmap="RdBu_r",              # red = positive, blue = negative
    vmin=-max_abs,              # force symmetry
    vmax=max_abs,
    legend=True,
    legend_kwds={"caption": "Tridico − Occhiuto (%)"},
    tooltip=[
        "SEZIONE",
        "TRIDICO_%",
        "OCCHIUTO_%",
        "TRI_vs_OCC_margin_%"
    ],
    tooltip_kwds={"sticky": True},
    style_kwds={"weight": 0.8, "fillOpacity": 0.6},
    tiles="CartoDB positron"
)

m_margin.save("mappe\\tridico_vs_occhiuto_margin.html")



In [13]:
os.listdir("mappe")

['affluenza_map.html',
 'tridico_share_map.html',
 'tridico_vs_occhiuto_margin.html']

In [26]:
m_affluenza.show_in_browser()

Your map should have been opened in your browser automatically.
Press ctrl+c to return.


In [22]:
dplot.columns

Index(['SEZIONE', 'Shape_Area', 'sez_cens', 'unassigned', 'geometry',
       'OCCHIUTO', 'TOSCANO', 'TRIDICO', 'TOTALE', 'AFFLUENZA'],
      dtype='object')